# Test Map Display in Jupyter Notebooks

This notebook tests the issue reported in GitHub #23 regarding the polar map module not displaying properly in Jupyter notebooks without cell magic.

According to the issue:
- ✅ Works: Using `%%html` cell magic 
- ❌ Doesn't work (or didn't): Using Python with `IPython.display.HTML`

The current `polar.html` implementation now supports URL parameters, which should fix the issue.

## Test 1: HTML Cell Magic (Should Work)

First, let's test with the `%%html` cell magic, which was reportedly working:

In [ ]:
%%html 
<iframe 
    src="https://storage.googleapis.com/opr_stac/map/polar.html?pole=south&parquetFiles=[%22https://storage.googleapis.com/opr_stac/testing/2010_Antarctica_DC8.parquet%22]&defaultZoom=3" 
    width="100%" 
    height="600"
    frameborder="0"
    style="border: 1px solid #ccc; border-radius: 5px;">
</iframe>

## Test 2: Python HTML Display (Previously Broken)

Now let's test using Python's `IPython.display.HTML`, which reportedly didn't work. 
With the URL parameter support in `polar.html`, this should now work:

In [ ]:
from IPython.display import IFrame, HTML
import json

def create_polar_map(pole='south', parquet_files=None, height=600):
    """
    Create an embedded polar map.
    
    Parameters:
    -----------
    pole : str
        'north' or 'south' for Arctic or Antarctic projection
    parquet_files : list
        List of parquet file paths to display
    height : int
        Height of the map in pixels
    """
    if parquet_files is None:
        # Use a test file from GCS
        parquet_files = ['https://storage.googleapis.com/opr_stac/testing/2010_Antarctica_DC8.parquet']
    
    # Create URL parameters
    params = {
        'pole': pole,
        'parquetFiles': json.dumps(parquet_files),
        'defaultZoom': 3
    }
    
    # Build query string
    query_string = '&'.join(f"{k}={v}" for k, v in params.items())
    
    # Create HTML with URL parameters
    html = f'''
    <iframe 
        src="https://storage.googleapis.com/opr_stac/map/polar.html?{query_string}"
        width="100%" 
        height="{height}"
        frameborder="0"
        style="border: 1px solid #ccc; border-radius: 5px;">
    </iframe>
    '''
    
    return HTML(html)

# Display the map
create_polar_map(pole='south', height=500)

## Test 3: Multiple Files with Different Colors

Let's test the newer `fileGroups` parameter which allows displaying multiple files with different colors:

In [ ]:
def create_polar_map_with_groups(pole='south', file_groups=None, height=600):
    """
    Create an embedded polar map with file groups.
    
    Parameters:
    -----------
    pole : str
        'north' or 'south' for Arctic or Antarctic projection
    file_groups : list of dict
        List of file groups, each with 'files' and 'color' keys
    height : int
        Height of the map in pixels
    """
    if file_groups is None:
        # Use test files with different colors
        file_groups = [
            {
                'files': ['https://storage.googleapis.com/opr_stac/testing/2010_Antarctica_DC8.parquet'],
                'color': 'red'
            },
            {
                'files': ['https://storage.googleapis.com/opr_stac/testing/2016_Antarctica_DC8.parquet'],
                'color': 'blue'
            }
        ]
    
    # Create URL parameters
    params = {
        'pole': pole,
        'fileGroups': json.dumps(file_groups),
        'defaultZoom': 3
    }
    
    # Build query string
    query_string = '&'.join(f"{k}={v}" for k, v in params.items())
    
    # Create HTML with URL parameters
    html = f'''
    <iframe 
        src="https://storage.googleapis.com/opr_stac/map/polar.html?{query_string}"
        width="100%" 
        height="{height}"
        frameborder="0"
        style="border: 1px solid #ccc; border-radius: 5px;">
    </iframe>
    '''
    
    return HTML(html)

# Display the map with multiple file groups
create_polar_map_with_groups(pole='south', height=500)

## Test 4: Using the Legacy CONFIG Object Method

Let's also test the legacy method using the CONFIG object set via JavaScript:

In [ ]:
def create_polar_map_legacy(pole='south', parquet_files=None, height=600):
    """
    Create an embedded polar map using the legacy CONFIG method.
    
    Parameters:
    -----------
    pole : str
        'north' or 'south' for Arctic or Antarctic projection
    parquet_files : list
        List of parquet file paths to display
    height : int
        Height of the map in pixels
    """
    if parquet_files is None:
        parquet_files = ['https://storage.googleapis.com/opr_stac/testing/2010_Antarctica_DC8.parquet']
    
    config = {
        'pole': pole,
        'parquetFiles': parquet_files,
        'defaultZoom': 3
    }
    
    # Create HTML with embedded configuration
    html = f'''
    <iframe 
        src="https://storage.googleapis.com/opr_stac/map/polar.html"
        width="100%" 
        height="{height}"
        frameborder="0"
        style="border: 1px solid #ccc; border-radius: 5px;"
        onload="this.contentWindow.CONFIG = {json.dumps(config)}">
    </iframe>
    '''
    
    return HTML(html)

# Display the map using legacy CONFIG method
create_polar_map_legacy(pole='south', height=500)

## Test 5: Arctic (North Pole) Example

Let's also test with Arctic data to ensure the north pole projection works:

In [ ]:
# Test with north pole (Arctic)
def create_arctic_map():
    # Note: Using Antarctica data but displaying on north pole projection for testing
    # In production, you'd use actual Arctic/Greenland data files
    parquet_files = ['https://storage.googleapis.com/opr_stac/testing/2010_Antarctica_DC8.parquet']
    
    params = {
        'pole': 'north',
        'parquetFiles': json.dumps(parquet_files),
        'defaultZoom': 3
    }
    
    query_string = '&'.join(f"{k}={v}" for k, v in params.items())
    
    html = f'''
    <iframe 
        src="https://storage.googleapis.com/opr_stac/map/polar.html?{query_string}"
        width="100%" 
        height="500"
        frameborder="0"
        style="border: 1px solid #ccc; border-radius: 5px;">
    </iframe>
    '''
    
    return HTML(html)

create_arctic_map()

## Summary

With the current implementation of `polar.html` that supports URL parameters:

1. **✅ HTML Cell Magic** - Should continue to work as before
2. **✅ Python HTML Display** - Now works with URL parameters (previously broken)
3. **✅ File Groups** - New feature for displaying multiple files with different colors
4. **✅ Legacy CONFIG Method** - Should still work for backward compatibility
5. **✅ Both Poles** - Support for both Arctic and Antarctic projections

The issue reported in GitHub #23 should now be resolved since the map can receive configuration via URL parameters, which works correctly when generated from Python code.